# Weave @SambaNova

This notebook demonstrates how to use [W&B Weave](https://weave-docs.wandb.ai/) with [SambaNova](https://sambanova.ai/) as your fastest model provider of choice for open source models.


In order to use `Weave` with `SambaNova`, you need to set the environment variable `SAMBANOVA_API_KEY`: Your API key for accessing your SambaNova instance. You can create your API key [here](https://cloud.sambanova.ai/apis).

To get started, simply call `weave.init()` at the beginning of your script, with the project name as attribute.

`Weave` ops make results reproducible by automatically versioning code as you experiment, and they capture their inputs and outputs.

Simply create a function decorated with `@weave.op()` that calls into each completion function and Weave will track the inputs and outputs for you. 

By using the `weave.Model` class, you can capture and organize the experimental details of your app like your system prompt or the model you're using.
This helps organize and compare different iterations of your app.

In [26]:
import os
from typing import Any, Optional

import weave
from dotenv import load_dotenv

In [3]:
# If you have SAMBANOVA_API_KEY in your .env file
SAMBANOVA_API_KEY = os.getenv('SAMBANOVA_API_KEY')

# Initialize Weave with your project name
model = 'Meta-Llama-3.3-70B-Instruct'

## Via LangChain

`Weave` is designed to make tracking and logging all calls made through the `LangChain` Python library effortless, after `weave.init()` is called.

You can access all the features of the `LangChain` + `Weave` integration, by using our `LangChain` chat object, `langchain_sambanova.ChatSambaNovaCloud`.

For more details on all the `Weave` features supported by `LangChain`, please refer to [Weave @LangChain](https://weave-docs.wandb.ai/guides/integrations/langchain/).

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain_sambanova import ChatSambaNovaCloud

# Initialize Weave project
weave.init('weave_integration_sambanova_langchain')

/Users/francescar/Documents/ai-starter-kit/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Logged in as Weights & Biases user: francescar.
View Weave data at https://sambanova.wandb.io/ai-solutions-team/weave_integration_sambanova_langchain/weave


### Simple call

In [5]:
# The LangChain SambaNova Chat object
llm = ChatSambaNovaCloud(
    model=model,
    temperature=0.7,
    top_p=0.95,
)

# The prompt template
prompt = PromptTemplate.from_template('1 + {number} = ')

# The LLM chain
llm_chain = prompt | llm

In [6]:
# Invoke the LLM on the prompt
output = llm_chain.invoke({'number': 2})

print(output.content)

1 + 2 = 3


🍩 https://sambanova.wandb.io/ai-solutions-team/weave_integration_sambanova_langchain/r/call/019688f6-fb50-7e92-a43a-6ed5af7c15f6
🍩 https://sambanova.wandb.io/ai-solutions-team/weave_integration_sambanova_langchain/r/call/019688f6-fecb-7f02-a54e-9ff4db50d84b


### Tracking Call Metadata
To track metadata from your `LangChain` calls, you can use the `weave.attributes` context manager. This context manager allows you to set custom metadata for a specific block of code, such as a chain or a single request.

In [7]:
# The LLM chain with Weave attributes
with weave.attributes({'number_to_increment': 'value'}):
    output = llm_chain.invoke({'number': 2})

print(output.content)

1 + 2 = 3


## Via LiteLLM

`Weave` automatically tracks and logs LLM calls made via LiteLLM, after `weave.init()` is called.

You can access all the features of the `Weave` + `LiteLLM` integration, by specifying the `SambaNova` model name in the `LiteLLM` constructor, as explained in [LiteLLM @SambaNova](https://docs.litellm.ai/docs/providers/sambanova).

For more details on all the `Weave` features supported by `LiteLLM`, please refer to [Weave @LiteLLM](https://weave-docs.wandb.ai/guides/integrations/litellm).

In [8]:
import litellm

# Initialize Weave project
weave.init('weave_integration_sambanova_litellm')
model_litellm = 'sambanova/' + model

Logged in as Weights & Biases user: francescar.
View Weave data at https://sambanova.wandb.io/ai-solutions-team/weave_integration_sambanova_litellm/weave


### Simple call

In [9]:
question = "Translate 'Hello, how are you?' to French."
response = litellm.completion(
    model=model_litellm,
    messages=[{'role': 'user', 'content': question}],
    max_tokens=1024,
)
print(response.choices[0].message.content)

/Users/francescar/Documents/ai-starter-kit/.venv/lib/python3.11/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `int` - serialized value may not be as expected [input_value=1746054613.439177, input_type=float])
  return self.__pydantic_serializer__.to_python(
Error capturing call output:
Traceback (most recent call last):
  File "/Users/francescar/Documents/ai-starter-kit/.venv/lib/python3.11/site-packages/weave/trace/op.py", line 424, in _call_sync_func
    res = on_output(res)
          ^^^^^^^^^^^^^^
  File "/Users/francescar/Documents/ai-starter-kit/.venv/lib/python3.11/site-packages/weave/trace/op.py", line 407, in on_output
    return handler(output, finish, call.inputs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/francescar/Documents/ai-starter-kit/.venv/lib/python3.11/site-packages/weave/trace/op.py", line 1094, in on_output
    wrapped_on_finish(value)
  File "/Users/francescar/Do

🍩 https://sambanova.wandb.io/ai-solutions-team/weave_integration_sambanova_litellm/r/call/019688f7-051e-7811-967a-1cb64377283e
The translation of 'Hello, how are you?' to French is:

"Bonjour, comment allez-vous?"

This is a formal way of asking. If you want to ask someone you're familiar with, you can use:

"Salut, comment ça va?"

Both of these phrases are commonly used in French to ask about someone's well-being.


🍩 https://sambanova.wandb.io/ai-solutions-team/weave_integration_sambanova_litellm/r/call/019688f7-0a9f-71f1-8cb3-680c79a126b0
🍩 https://sambanova.wandb.io/ai-solutions-team/weave_integration_sambanova_litellm/r/call/019688f7-0c7f-78a0-97b1-0179538f996c
🍩 https://sambanova.wandb.io/ai-solutions-team/weave_integration_sambanova_litellm/r/call/019688fa-3ac6-7b92-b42c-797180c511a7
🍩 https://sambanova.wandb.io/ai-solutions-team/weave_integration_sambanova_litellm/r/call/019688fa-8b42-7312-b9a0-69d7ac8a0cfb


### @weave.op

In [ ]:
@weave.op()
def translate_litellm(model: str, text: str, target_language: str) -> Any:
    response = litellm.completion(
        model=model, messages=[{'role': 'user', 'content': f"Translate '{text}' to {target_language}"}], max_tokens=1024
    )
    return response.choices[0].message.content

In [ ]:
translate_litellm(model_litellm, 'Hello, how are you?', 'French')

/Users/francescar/Documents/ai-starter-kit/.venv/lib/python3.11/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `int` - serialized value may not be as expected [input_value=1746054614.2416449, input_type=float])
  return self.__pydantic_serializer__.to_python(


'The translation of \'Hello, how are you?\' to French is:\n\n"Bonjour, comment allez-vous?"\n\nThis is a formal way of asking. If you want to ask someone you know, you can use the informal version:\n\n"Salut, comment ça va?"'

### weave.Model

In [12]:
class LiteLLMTranslatorModel(weave.Model):  # type: ignore
    model: str
    temperature: float

    @weave.op()  # type: ignore
    def predict(self, text: str, target_language: str) -> Any:
        response = litellm.completion(
            model=self.model,
            messages=[
                {'role': 'system', 'content': f'You are a translator. Translate the given text to {target_language}.'},
                {'role': 'user', 'content': text},
            ],
            max_tokens=1024,
            temperature=self.temperature,
        )
        return response.choices[0].message.content

In [15]:
# Create an instance of the translator weave.Model
translator = LiteLLMTranslatorModel(model=model_litellm, temperature=0.3)

english_text = 'Hello, how are you today?'
french_text = translator.predict(english_text, 'French')

print(french_text)

Bonjour, comment allez-vous aujourd'hui ?


/Users/francescar/Documents/ai-starter-kit/.venv/lib/python3.11/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `int` - serialized value may not be as expected [input_value=1746054844.0954764, input_type=float])
  return self.__pydantic_serializer__.to_python(


## Via the OpenAI SDK

`SambaNova` supports the `OpenAI` SDK compatibility ([docs](https://docs.sambanova.ai/cloud/docs/capabilities/openai-compatibility)) which `Weave` automatically detects and integrates with.

To switch to using the SambaNova API, simply switch out the `api_key` to your SambaNova API key, `base_url` to https://api.sambanova.ai/v1, and `model` to one of our chat models.

In [ ]:
from openai import OpenAI

# Initialize Weave project
weave.init('weave_integration_sambanova_openai_sdk')

# SambaNova URL: https://api.sambanova.ai/v1
SAMBANOVA_URL = os.getenv('SAMBANOVA_URL')

# Set the sambanova client
sambanova_client = OpenAI(base_url=SAMBANOVA_URL, api_key=SAMBANOVA_API_KEY)

Logged in as Weights & Biases user: francescar.
View Weave data at https://sambanova.wandb.io/ai-solutions-team/weave_integration_sambanova_openai_sdk/weave


### Simple call

In [ ]:
# Create a grammar corrector
response = sambanova_client.chat.completions.create(
    model=model,
    messages=[
        {'role': 'system', 'content': 'You are a grammar checker, correct the following user input.'},
        {'role': 'user', 'content': 'That was so easy, it was a piece of pie!'}],
    temperature=0,
)

print(response.choices[0].message.content)

The corrected sentence is:

"That was so easy, it was a piece of cake!"

The original phrase "a piece of pie" is not a common idiomatic expression. The correct idiom is "a piece of cake," which means something is very easy to do. However, it's worth noting that the original sentence "a piece of pie" is still grammatically correct and can be used in a literal sense, but it doesn't convey the same idiomatic meaning as "a piece of cake."


/Users/francescar/Documents/ai-starter-kit/.venv/lib/python3.11/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `int` - serialized value may not be as expected [input_value=1746056030.9465058, input_type=float])
  return self.__pydantic_serializer__.to_python(


### @weave.op

In [ ]:
@weave.op()
def correct_grammar(model: str, system_prompt: str, user_prompt: str) -> Optional[str]:
    """Correct the grammar of a text."""
    
    response = sambanova_client.chat.completions.create(
        model=model,
        messages=[{'role': 'system', 'content': system_prompt}, {'role': 'user', 'content': user_prompt}],
        temperature=0,
    )
    return response.choices[0].message.content

response = correct_grammar(model, 'You are a grammar checker, correct the following user input.', 'That was so easy, it was a piece of pie!')

print(response)

### weave.Model

In [ ]:
class GrammarCorrectorModel(weave.Model):
    model: str
    system_message: str

    @weave.op()
    def predict(self, user_input: str) -> Optional[str]:
        """Correct the grammar of a text."""

        response = sambanova_client.chat.completions.create(
            model=self.model,
            messages=[{'role': 'system', 'content': self.system_message}, {'role': 'user', 'content': user_input}],
            temperature=0,
        )
        return response.choices[0].message.content

# Correct grammar
corrector = GrammarCorrectorModel(
    model=model, system_message='You are a grammar checker, correct the following user input.'
)
result = corrector.predict('That was so easy, it was a piece of pie!')
print(result)